In [ ]:
!pip install -q gradio
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from pandas.core.arrays.numeric import T
def multilnreg(x,y):
  reg = linear_model.LinearRegression()
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.40)
  fitting=reg.fit(X_train,y_train)
  predict=reg.predict(X_test)
  rmse = np.sqrt(mean_squared_error(y_test,predict))
  r_squared = r2_score(y_test,predict)
  X = sm.add_constant(x) 
  model = sm.OLS(y, X).fit()
  predictions = model.predict(X)
  summary=model.summary()
  return str(rmse),str(r_squared),str(summary)
def multilinregtest(f1):
  def updownsplitTrend(xsplit,ysplit,x,y):
      l=xsplit
      xmid=int(xsplit/2)
      ymid=int(ysplit/2)
      i=0
      inc=0
      dec=0
      while (xsplit>3 and ysplit>3) and i<l-1:
          if (x[i]<x[xsplit-1] and y[i]<y[ysplit-1]) or (x[i+1]<x[xsplit-2] and y[i+1]<y[ysplit-2]) or (x[i+2]<x[xsplit-3] and y[i+2]<y[ysplit-3]):
              if (x[xmid-4]<x[xmid+3] and y[ymid-4]<y[ymid+3]) or (x[xmid-3]<x[xmid+2] and y[ymid-3]<y[ymid+2]) or (x[xmid-2]<x[xmid+1] and y[ymid-2]<y[ymid+1]) or (x[xmid-1]<x[xmid] and y[ymid-1]<y[ymid]):
                  inc+=1
          elif (x[i]>x[xsplit-1] and y[i]>y[ysplit-1]) or (x[i+1]>x[xsplit-2] and y[i+1]>y[ysplit-2]) or (x[i+2]>x[xsplit-3] and y[i+2]>y[ysplit-3]):
              if (x[xmid-4]>x[xmid+3] and y[ymid-4]>y[ymid+3]) or (x[xmid-3]>x[xmid+2] and y[ymid-3]>y[ymid+2]) or (x[xmid-2]>x[xmid+1] and y[ymid-2]>y[ymid+1]) or (x[xmid-1]>x[xmid] and y[ymid-1]>y[ymid]):
                  dec+=1
          xsplit-=1
          ysplit-=1
      if inc>0:
          # "Increasing Trend"
          t=True
      elif dec>0:
          # "Decreasing Trend"
          t=True
      else:
          # "Inconsistent updown trend"
          t=False
      return t
          
  def splitTrend(x,y,xsplit,ysplit):
      up=0
      down=0
      updown=0
      negativecorrelation=0
      upstreak=0
      downstreak=0
      ncstreak=0
      l1=len(x)
      l2=len(y)
      l=l1
      i=0
      while l>0 and i<(l1-1):
      #check dtype of x and y when returning from df then get into streaks
          if (x[i+1]>x[i] and y[i+1]>y[i]):
              if i>=1:
                  upstreak+=1
                  up+=1
                  downstreak=0
                  ncstreak=0
          elif (x[i+1]<x[i] and y[i+1]<y[i]) :
              negativecorrelation+=1
              ncstreak+=1
              up+=1
              upstreak=0
              downstreak=0
          elif (x[i+1]>x[i] and y[i+1]<y[i]) :
              negativecorrelation+=1
              ncstreak+=1
              down+=1
              upstreak=0
              downstreak=0
          elif  (x[i+1]<x[i] and y[i+1]>y[i]):
              if i>=1:
                  downstreak+=1
                  down+=1
                  upstreak=0
          
          if i==(l1-2):
              t=trend(up,down,updown,negativecorrelation,l1,l2,x,y,xsplit,ysplit)
          l=l-1
          i=i+1
      return t

  def Range(n,p,k):
      count=0
      if n%2==0:
          if n<10000:
              k=n/10
              p=k/10
          elif n>10000:
              count+=1
              n=n/2
              Range(n,0,0)
      else:
          count-=1
          n=n-1
          Range(n,0,0)
      return int(n),int(p),int(k),int(count)

  def updowntrend(xlen,ylen,xsplit,ysplit):
      while (xlen==ylen and xlen>0 and ylen>0):
          if xlen%2==0:
              if xlen>500:
                  xlen=xlen/2
                  ylen=ylen/2
                  xsplit=xlen/10
                  ysplit=ylen/10
                  break
              else:
                  xsplit=xlen/10
                  ysplit=ylen/10
                  break
          else:
              xlen-=1
              ylen-=1
              updowntrend(xlen,ylen,xsplit,ysplit)
      return xsplit,ysplit
      

  def trend(up,down,updown,negativecorrelation,l1,l2,x,y,xsplit,ysplit):
      if (up-down)in [-1,0,1]:
          updown+=1
      elif (up-negativecorrelation) in [-1,0,1]:
          updown+=1
      if updown>0:
          # "updown trend exists"
          if l1<100:
              t=updownsplitTrend(l1,l2,x,y)
          else:
              xsplit,ysplit=updowntrend(l1,l2,xsplit,ysplit)
              t=updownsplitTrend(int(xsplit),int(ysplit),x,y)
      elif up>(l1/2):
              # "Plain Increasing trend"
              t=True
      elif down>(l1):
              # "Plain Decreasing trend"
              t=True
      else:
          # "No co-realtion"
          t=False
      return t

  df=pd.read_csv(f1)
  n=df.shape[0]
  k=0
  p=0
  if n>10:
      n1,p,k,c=Range(n,p,k)

  col=list(df.columns)
  rcol=[]
  for a in col:
      if df[a].dtype==int or df[a].dtype==float or df[a].dtype==np.int64 or df[a].dtype==np.float64 or df[a].dtype==np.int32 or df[a].dtype==np.float32:
          rcol.append(a)
  def tvalcheck(rcol,colx,coly,df,x,y,xsplit,ysplit):
      i=0
      tval=0 
      ln=len(rcol)
      colx=colx
      coly=coly
      xcollist=[colx]
      for cl in rcol:
        if i>=(ln-1):
          break
        if coly==cl or colx==cl:
            i+=1
            continue
        elif coly!=cl:
          if i>=(ln-1):
            break
          else:
            colx=rcol[i]
            x=df[colx]
            t=splitTrend(x,y,xsplit,ysplit)
            i=i+1
        if t==True:
          xcollist.append(colx)
        i+=1
      if len(xcollist)>=2:
        return xcollist,True
      else:
        return [],False    
  def call(rcol,df,xsplit,ysplit):
      i=0
      tval=0 
      ln=len(rcol)
      rmseind=0
      rmse=0
      rsq=0
      tcheck=False
      while i<(ln-1):
          colx=rcol[i]
          coly=rcol[i+1]
          x=df[colx]
          y=df[coly]
          t=splitTrend(x,y,xsplit,ysplit)
          if t==True:
            tval+=1
          if tval>=1 and ln>2:
            xcollist,tcheck=tvalcheck(rcol,colx,coly,df,x,y,xsplit,ysplit)
          if tcheck==True:
            # x=df[xcollist]
            return xcollist,coly
            # rmse,rsq,summary=multilnreg(x,np.array(y))
          i+=1
      return xcollist,coly

  xsplit=0
  ysplit=0
  xcol,ycol=call(rcol,df,xsplit,ysplit)
  if xcol==[] and ycol=='':
    return False,[],[]
  else:
    x=df[xcol]
    y=df.loc[:,ycol]
    return True,x,np.array(y)

import gradio as gr
def multi_linreg(file1):
  v,x,y=multilinregtest(file1.name)
  if v==True:
    rmse,rsq,summary=multilnreg(x,y)
    return "RMSE:"+rmse+"\nR-SQUARED:"+rsq+"\nSummary"+summary
  else:
    return "0"

inf=gr.Interface(multi_linreg,inputs="file",outputs="text",capture_session=True)
inf.launch(share=True)

#tester code
# def linreg(file1):
#   v,x,y=multilinregtest(file1)
#   if v==True:
#     rmse,rsq,summary=multilnreg(x,y)
#     return "RMSE:"+rmse+"R-SQUARED:"+rsq+"Summary"+summary
#   else:
#     return "0"
# linreg("xyz corp.csv")
#code works but check why it is stalling for a long time

     |████████████████████████████████| 2.9 MB 5.0 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 253 kB 57.4 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 1.1 MB 43.3 MB/s 
     |████████████████████████████████| 211 kB 56.5 MB/s 
     |████████████████████████████████| 84 kB 2.9 MB/s 
     |████████████████████████████████| 2.0 MB 48.9 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 271 kB 43.0 MB/s 
     |████████████████████████████████| 144 kB 48.0 MB/s 
     |████████████████████████████████| 10.9 MB 28.7 MB/s 
     |████████████████████████████████| 58 kB 2.2 MB/s 
     |████████████████████████████████| 79 kB 5.0 MB/s 
     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 856 kB 44.7 MB/s 
     |████████████████████████████████| 61 kB 418 kB/s 
     |████████████████████████

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.7/dist-packages/gradio/interface.py:248: UserWarning: The `capture_session` parameter in the `Interface` is deprecated and may be removed in the future.
  "The `capture_session` parameter in the `Interface`"


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://18395.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x7f61a7320ad0>,
 'http://127.0.0.1:7860/',
 'https://18395.gradio.app')